In [1]:
import numpy as np
import pandas as pd

import os
import pickle

import nltk
import gensim

from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN
from sklearn.model_selection import train_test_split
import theano
theano.config.optimizer="None"

Using Theano backend.


In [2]:
with open('kramer_source_list_w2v.pickle','rb') as file:
    kramer_source_list_w2v = pickle.load(file)

In [3]:
with open('kramer_response_list_w2v.pickle','rb') as file:
    kramer_response_list_w2v = pickle.load(file)

In [4]:
# Setup nltk corpora path and Google Word2Vec location
nltk_path = os.sep.join([os.environ['HOME'], 'nltk_data'])
google_vec_file = '/Users/warren/Data_Science/Metis/github/project-kojak/GoogleNews-vectors-negative300.bin.gz'

nltk.data.path.insert(0, nltk_path)

In [5]:
%%time

word2vec = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

CPU times: user 2min 44s, sys: 5.76 s, total: 2min 50s
Wall time: 2min 53s


In [6]:
# Convert prompts and responses to arrays
vec_x = np.array(kramer_source_list_w2v, dtype = np.float64)
vec_y = np.array(kramer_response_list_w2v, dtype = np.float64)

In [7]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(vec_x, vec_y, test_size = 0.2, random_state = 1)

In [ ]:
# Model and training parameters
LAYER_NUM = 4
BATCH_SIZE = 10
NUM_EPOCH = 10
PRED_SIZE = 10

In [ ]:
# Create model
# Add additional layers after the LSTMs?
model = Sequential()

for i in range(LAYER_NUM - 1):
    model.add(LSTM(output_dim = 300,
                   input_shape = x_train.shape[1:],
                   return_sequences = True,
                   init = 'glorot_normal',
                   inner_init = 'glorot_normal',
                   activation = 'sigmoid'))

model.compile(loss = 'cosine_proximity', optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
# Train model
# 10, 40, 450, 500, 500, ... (until cumsum == 5000?)
nb_epoch = 0

while True:
    print('\n\n')
    model.fit(x_train, y_train,
              batch_size = BATCH_SIZE,
              nb_epoch = NUM_EPOCH,
              validation_data=(x_test, y_test),
              verbose = 1)

    nb_epoch += NUM_EPOCH
    
    if nb_epoch % 10 == 0:
        pprint(model.predict(x_test[:PRED_SIZE]) #generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_layers_{}_epoch_{}.hdf5'.format(LAYER_NUM, nb_epoch))


In [ ]:
#[word2vec.most_similar([predictions[10][i]])[0] for i in range(28)]